In [1]:
%pip install tiktoken pymupdf -q

Note: you may need to restart the kernel to use updated packages.


# 필수 패키지 설치

# document_processer 테스트

In [5]:
# 환경 설정 및 모듈 임포트
import sys
from pathlib import Path

# 프로젝트 루트를 sys.path에 추가
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.processors.document_processor import DocumentProcessor
from src.db.documents_db import DocumentsDB
from src.utils.logging_config import get_logger

logger = get_logger(__name__)
logger.info("테스트 환경 설정 완료")

2025-11-08 22:35:20 [I] __main__ - 테스트 환경 설정 완료


In [11]:
def test_real_pdf_if_available() -> None:
    """실제 PDF 파일이 있는 경우 테스트"""
    from pathlib import Path
    
    # data/raw 디렉토리에서 PDF 파일 검색
    raw_dir = project_root / "data" / "raw"
    pdf_files = list(raw_dir.glob("*.pdf")) if raw_dir.exists() else []
    
    if not pdf_files:
        logger.warning("⚠ data/raw 디렉토리에 PDF 파일이 없습니다.")
        logger.info("테스트를 건너뜁니다. PDF 파일을 추가하려면:")
        logger.info(f"  경로: {raw_dir}")
        return
    
    logger.info(f"발견된 PDF 파일: {len(pdf_files)}개")
    
    # 첫 번째 PDF 파일 처리
    pdf_path = pdf_files[0]
    logger.info(f"처리 중: {pdf_path.name}")
    
    try:
        processor = DocumentProcessor(db_path=str(project_root / "data" / "test_documents.db"))
        file_hash = processor.process_pdf(str(pdf_path))
        
        if file_hash:
            logger.info(f"✓ PDF 처리 성공: {file_hash[:16]}...")
        else:
            logger.error("✗ PDF 처리 실패 (PyMuPDF 미설치 가능성)")
            
    except Exception as e:
        logger.error(f"✗ PDF 처리 중 오류: {e}")

# 실행
test_real_pdf_if_available()

2025-11-08 22:37:02 [W] __main__ - ⚠ data/raw 디렉토리에 PDF 파일이 없습니다.
2025-11-08 22:37:02 [I] __main__ - 테스트를 건너뜁니다. PDF 파일을 추가하려면:
2025-11-08 22:37:02 [I] __main__ -   경로: d:\GoogleDrive\codeit_ai_g2b_search\data\raw
2025-11-08 22:37:02 [I] __main__ - 테스트를 건너뜁니다. PDF 파일을 추가하려면:
2025-11-08 22:37:02 [I] __main__ -   경로: d:\GoogleDrive\codeit_ai_g2b_search\data\raw


## 5. 실제 PDF 파일 테스트 (선택사항)

In [10]:
def verify_database_content() -> None:
    """DB 내용 확인 및 검증"""
    import sqlite3
    
    test_db_path = project_root / "data" / "test_documents.db"
    
    with sqlite3.connect(test_db_path) as conn:
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        # 1. 파일 정보 조회
        cursor.execute("SELECT * FROM file_info")
        files = cursor.fetchall()
        logger.info(f"\n=== 파일 정보 ({len(files)}건) ===")
        for row in files:
            logger.info(f"  - {row['file_name']}: {row['total_pages']}페이지, {row['total_tokens']}토큰")
        
        # 2. 페이지 데이터 조회
        cursor.execute("""
            SELECT file_hash, page_number, token_count, is_empty 
            FROM page_data 
            ORDER BY file_hash, page_number
        """)
        pages = cursor.fetchall()
        logger.info(f"\n=== 페이지 데이터 ({len(pages)}건) ===")
        
        # 파일별로 그룹화
        from collections import defaultdict
        pages_by_file = defaultdict(list)
        for row in pages:
            pages_by_file[row['file_hash']].append(row)
        
        for file_hash, file_pages in pages_by_file.items():
            logger.info(f"  파일: {file_hash[:16]}...")
            for page in file_pages:
                status = "빈페이지" if page['is_empty'] else "정상"
                logger.info(f"    - 페이지 {page['page_number']}: {page['token_count']}토큰 ({status})")
        
        # 3. 통계 조회
        db = DocumentsDB(str(test_db_path))
        stats = db.get_document_stats()
        logger.info(f"\n=== 전체 통계 ===")
        logger.info(f"  총 파일: {stats['total_files']}개")
        logger.info(f"  총 페이지: {stats['total_pages']}개")
        logger.info(f"  총 토큰: {stats['total_tokens']:,}개")
        logger.info(f"  총 크기: {stats['total_size_mb']}MB")
        
        logger.info("\n✓ DB 검증 완료")

# 실행
verify_database_content()

2025-11-08 22:36:43 [I] __main__ - 
=== 파일 정보 (2건) ===
2025-11-08 22:36:43 [I] __main__ -   - test_document.pdf: 3페이지, 1200토큰
2025-11-08 22:36:43 [I] __main__ -   - test_dummy.pdf: 4페이지, 69토큰
2025-11-08 22:36:43 [I] __main__ - 
=== 페이지 데이터 (7건) ===
2025-11-08 22:36:43 [I] __main__ -   파일: dummy_pdf_hash_6...
2025-11-08 22:36:43 [I] __main__ -   - test_document.pdf: 3페이지, 1200토큰
2025-11-08 22:36:43 [I] __main__ -   - test_dummy.pdf: 4페이지, 69토큰
2025-11-08 22:36:43 [I] __main__ - 
=== 페이지 데이터 (7건) ===
2025-11-08 22:36:43 [I] __main__ -   파일: dummy_pdf_hash_6...
2025-11-08 22:36:43 [I] __main__ -     - 페이지 1: 22토큰 (정상)
2025-11-08 22:36:43 [I] __main__ -     - 페이지 2: 22토큰 (정상)
2025-11-08 22:36:43 [I] __main__ -     - 페이지 3: 6토큰 (빈페이지)
2025-11-08 22:36:43 [I] __main__ -     - 페이지 1: 22토큰 (정상)
2025-11-08 22:36:43 [I] __main__ -     - 페이지 2: 22토큰 (정상)
2025-11-08 22:36:43 [I] __main__ -     - 페이지 3: 6토큰 (빈페이지)
2025-11-08 22:36:43 [I] __main__ -     - 페이지 4: 19토큰 (정상)
2025-11-08 22:36:43 [I] __m

## 4. DB 조회 및 검증

In [9]:
def test_pdf_processing_simulation() -> None:
    """PDF 처리 로직 시뮬레이션 (PyMuPDF 없이)"""
    
    # 실제 PDF 대신 더미 데이터로 로직 테스트
    test_db_path = project_root / "data" / "test_documents.db"
    processor = DocumentProcessor(db_path=str(test_db_path))
    
    # PDF 페이지 시뮬레이션
    dummy_pages = [
        "# 제목 페이지\n\n이것은 첫 번째 페이지입니다.",
        "# 본문 1\n\n두 번째 페이지 내용입니다.",
        "",  # 빈 페이지
        "# 결론\n\n마지막 페이지입니다."
    ]
    
    # 파일 정보 시뮬레이션
    dummy_hash = "dummy_pdf_hash_67890"
    dummy_filename = "test_dummy.pdf"
    
    logger.info(f"PDF 처리 시뮬레이션: {dummy_filename}")
    
    # 페이지별 처리
    all_content = []
    for page_num, page_text in enumerate(dummy_pages, start=1):
        is_empty = len(page_text.strip()) < 10
        
        if is_empty:
            page_content = "[빈 페이지]"
        else:
            page_content = f"--- 페이지 {page_num} ---\n\n{page_text}"
        
        all_content.append(page_content)
        
        # DB 저장
        token_count = processor.count_tokens(page_content)
        processor.docs_db.insert_page_data(
            file_hash=dummy_hash,
            page_number=page_num,
            markdown_content=page_content,
            token_count=token_count,
            is_empty=is_empty
        )
        logger.info(f"페이지 {page_num}: {token_count} 토큰, 빈페이지={is_empty}")
    
    # 전체 콘텐츠
    full_content = "\n\n".join(all_content)
    total_tokens = processor.count_tokens(full_content)
    
    # 파일 정보 저장
    processor.docs_db.insert_file_info(
        file_hash=dummy_hash,
        file_name=dummy_filename,
        total_pages=len(dummy_pages),
        file_size=len(full_content.encode('utf-8')),
        total_chars=len(full_content),
        total_tokens=total_tokens
    )
    
    logger.info(f"✓ PDF 시뮬레이션 완료: {len(dummy_pages)}페이지, {total_tokens}토큰")

# 테스트 실행
test_pdf_processing_simulation()

2025-11-08 22:36:33 [I] src.processors.document_processor - DocumentProcessor 초기화 완료 (DB: d:\GoogleDrive\codeit_ai_g2b_search\data\test_documents.db)
2025-11-08 22:36:33 [I] __main__ - PDF 처리 시뮬레이션: test_dummy.pdf
2025-11-08 22:36:33 [I] __main__ - PDF 처리 시뮬레이션: test_dummy.pdf
2025-11-08 22:36:33 [I] __main__ - 페이지 1: 22 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 2: 22 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 1: 22 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 2: 22 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 3: 6 토큰, 빈페이지=True
2025-11-08 22:36:33 [I] __main__ - 페이지 4: 19 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 3: 6 토큰, 빈페이지=True
2025-11-08 22:36:33 [I] __main__ - 페이지 4: 19 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - ✓ PDF 시뮬레이션 완료: 4페이지, 69토큰
2025-11-08 22:36:33 [I] __main__ - ✓ PDF 시뮬레이션 완료: 4페이지, 69토큰


## 3. PDF 처리 테스트 (PyMuPDF 없이 시뮬레이션)

In [8]:
def test_document_processor_utils() -> None:
    """DocumentProcessor 유틸리티 함수 테스트"""
    import tempfile
    
    # DocumentProcessor 초기화
    test_db_path = project_root / "data" / "test_documents.db"
    processor = DocumentProcessor(db_path=str(test_db_path))
    
    # 1. 임시 파일 생성
    with tempfile.NamedTemporaryFile(mode='w', suffix='.txt', delete=False, encoding='utf-8') as tf:
        test_content = "This is a test document for hash calculation."
        tf.write(test_content)
        temp_file_path = Path(tf.name)
    
    try:
        # 2. 파일 해시 계산
        file_hash = processor.calculate_file_hash(temp_file_path)
        logger.info(f"파일 해시: {file_hash[:16]}...")
        assert len(file_hash) == 64, "SHA-256 해시 길이 오류"
        
        # 3. 토큰 카운트
        token_count = processor.count_tokens(test_content)
        logger.info(f"토큰 수: {token_count}")
        assert token_count > 0, "토큰 카운트 오류"
        
        logger.info("✓ 유틸리티 함수 테스트 성공")
        
    finally:
        # 임시 파일 삭제
        if temp_file_path.exists():
            temp_file_path.unlink()

# 테스트 실행
test_document_processor_utils()

2025-11-08 22:36:26 [I] src.processors.document_processor - DocumentProcessor 초기화 완료 (DB: d:\GoogleDrive\codeit_ai_g2b_search\data\test_documents.db)
2025-11-08 22:36:26 [I] __main__ - 파일 해시: 258983484b09a700...
2025-11-08 22:36:26 [I] __main__ - 토큰 수: 9
2025-11-08 22:36:26 [I] __main__ - ✓ 유틸리티 함수 테스트 성공
2025-11-08 22:36:26 [I] __main__ - 파일 해시: 258983484b09a700...
2025-11-08 22:36:26 [I] __main__ - 토큰 수: 9
2025-11-08 22:36:26 [I] __main__ - ✓ 유틸리티 함수 테스트 성공


## 2. DocumentProcessor 해시 계산 및 토큰 카운트 테스트

In [7]:
def test_documents_db_crud() -> None:
    """DocumentsDB CRUD 테스트"""
    from datetime import datetime
    import sqlite3
    
    # 테스트용 DB 경로
    test_db_path = project_root / "data" / "test_documents.db"
    test_db_path.parent.mkdir(parents=True, exist_ok=True)
    
    # DB 초기화
    db = DocumentsDB(str(test_db_path))
    
    # 기존 테스트 데이터 삭제 (파일 삭제 대신)
    with sqlite3.connect(test_db_path) as conn:
        conn.execute("DELETE FROM page_data")
        conn.execute("DELETE FROM file_info")
        conn.commit()
    
    logger.info(f"테스트 DB 초기화: {test_db_path}")
    
    # 1. 파일 정보 삽입 (Create)
    test_hash = "test_hash_12345"
    success = db.insert_file_info(
        file_hash=test_hash,
        file_name="test_document.pdf",
        total_pages=3,
        file_size=1024 * 50,  # 50KB
        total_chars=5000,
        total_tokens=1200
    )
    logger.info(f"파일 정보 삽입: {success}")
    
    # 2. 페이지 데이터 삽입
    for page_num in range(1, 4):
        content = f"# 페이지 {page_num}\n\n테스트 콘텐츠입니다."
        db.insert_page_data(
            file_hash=test_hash,
            page_number=page_num,
            markdown_content=content,
            token_count=100 + page_num * 10,
            is_empty=False
        )
    logger.info("페이지 데이터 삽입 완료")
    
    # 3. 통계 조회 (Read)
    stats = db.get_document_stats()
    logger.info(f"문서 통계: {stats}")
    
    # 4. 데이터 확인
    assert stats['total_files'] >= 1, "파일 개수 불일치"
    assert stats['total_pages'] >= 3, "페이지 개수 불일치"
    
    logger.info("✓ CRUD 테스트 성공")
    return db, test_hash

# 테스트 실행
db, test_hash = test_documents_db_crud()

2025-11-08 22:36:20 [I] __main__ - 테스트 DB 초기화: d:\GoogleDrive\codeit_ai_g2b_search\data\test_documents.db
2025-11-08 22:36:20 [I] __main__ - 파일 정보 삽입: True
2025-11-08 22:36:20 [I] __main__ - 파일 정보 삽입: True
2025-11-08 22:36:20 [I] __main__ - 페이지 데이터 삽입 완료
2025-11-08 22:36:20 [I] __main__ - 문서 통계: {'total_files': 1, 'total_pages': 3, 'total_tokens': 1200, 'total_size_bytes': 51200, 'total_size_mb': 0.05}
2025-11-08 22:36:20 [I] __main__ - ✓ CRUD 테스트 성공
2025-11-08 22:36:20 [I] __main__ - 페이지 데이터 삽입 완료
2025-11-08 22:36:20 [I] __main__ - 문서 통계: {'total_files': 1, 'total_pages': 3, 'total_tokens': 1200, 'total_size_bytes': 51200, 'total_size_mb': 0.05}
2025-11-08 22:36:20 [I] __main__ - ✓ CRUD 테스트 성공


## 1. DocumentsDB 기본 CRUD 테스트

## 테스트 요약

위의 테스트들을 통해 다음을 확인했습니다:

1. **DocumentsDB CRUD**: 파일 정보/페이지 데이터 삽입 및 조회 정상 동작
2. **DocumentProcessor 유틸리티**: 파일 해시 계산, 토큰 카운트 정상
3. **PDF 처리 시뮬레이션**: 페이지별 콘텐츠 처리 로직 검증
4. **DB 검증**: 전체 통계 및 데이터 무결성 확인
5. **실제 PDF 처리**: PyMuPDF 사용 가능 시 자동 처리

**추가 테스트 방법**:
- `data/raw/` 디렉토리에 PDF 파일을 추가하면 실제 PDF 처리 테스트 자동 실행됩니다.
- HWP 파일 처리는 현재 `document_processor.py`에 구현되지 않았으므로 향후 추가 필요합니다.